**PART 3: Feature_Engineering**

Importing useful libraries at first

In [1]:
import pandas as pd
import numpy as np


import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import seaborn as sns
%matplotlib inline
pd.set_option('max_colwidth',200)
pd.set_option('max_columns',200)
SEED = 100

import warnings
warnings.filterwarnings("ignore")

import pickle
import time

import re
from bs4 import BeautifulSoup
import nltk
from nltk.tokenize import ToktokTokenizer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
from string import punctuation

import torch
from torch import nn ,cuda
from torch.utils.data import DataLoader,Dataset,RandomSampler, SequentialSampler

np.random.seed(SEED)

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Importing the database 

with open('/content/drive/MyDrive/thesis/preprocessed_dataframe.pkl', 'rb') as f:
    new_df = pickle.load(f)

In [4]:
new_df.head(5)

,Title,Body,Tags,no_of_tags
4,add script functionality .net applications,little game write c use database back-end trade card game want implement function card script mean essentially interface icard card class implement public class card056 icard contain function call...,"[c#, .net]",2
5,use nest class case,work collection class use video playback record one main class act like public interface methods like play stop pause record etc workhorse class video decode video encode learn existence nest clas...,[c++],1
6,homegrown consumption web service,write web service .net app ready consume see numerous examples homegrown code consume service oppose use auto generate methods visual studio create add web reference advantage,[.net],1
8,automatically update version number,would like version property application incremented build sure enable functionality visual studio 20052008 try specify assemblyversion 10 get exactly want also use settings file earlier attempt as...,[c#],1
10,connect database loop recordset c#,simplest way connect query database set record c#,[c#],1


In [5]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 44501 entries, 4 to 1263454
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Title       44501 non-null  object
 1   Body        44501 non-null  object
 2   Tags        44501 non-null  object
 3   no_of_tags  44501 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 1.7+ MB


In [6]:
df_body_tag = new_df[['Body','Tags']].copy()

In [7]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()
tags_encoded = mlb.fit_transform(df_body_tag['Tags'])
df_tags_encoded = pd.DataFrame(tags_encoded, columns=mlb.classes_)

In [8]:
df_tags_encoded.reset_index(drop=True, inplace=True)
df_body_tag.reset_index(drop=True, inplace=True)

In [9]:
df_merged = pd.concat([df_body_tag['Body'], df_tags_encoded], axis=1)
df_merged.head()

,Body,.net,android,c,c#,c++,css,html,ios,iphone,java,javascript,jquery,objective-c,php,python
0,little game write c use database back-end trade card game want implement function card script mean essentially interface icard card class implement public class card056 icard contain function call...,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0
1,work collection class use video playback record one main class act like public interface methods like play stop pause record etc workhorse class video decode video encode learn existence nest clas...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,write web service .net app ready consume see numerous examples homegrown code consume service oppose use auto generate methods visual studio create add web reference advantage,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,would like version property application incremented build sure enable functionality visual studio 20052008 try specify assemblyversion 10 get exactly want also use settings file earlier attempt as...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
4,simplest way connect query database set record c#,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


In [10]:
# create a list of tag columns
tag_cols = df_merged.filter(regex='^(?!.*Body).*$', axis=1).columns.tolist()

# zip tag columns to create a labels column
df_merged['labels'] = list(zip(*[df_merged[col].tolist() for col in tag_cols]))

In [14]:
df_merged['text'] = df_merged['Body']
df_merged.reset_index(drop=True, inplace=True)

In [15]:
df_merged.head()

,Body,.net,android,c,c#,c++,css,html,ios,iphone,java,javascript,jquery,objective-c,php,python,labels,text
0,little game write c use database back-end trade card game want implement function card script mean essentially interface icard card class implement public class card056 icard contain function call...,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,"(1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)",little game write c use database back-end trade card game want implement function card script mean essentially interface icard card class implement public class card056 icard contain function call...
1,work collection class use video playback record one main class act like public interface methods like play stop pause record etc workhorse class video decode video encode learn existence nest clas...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,"(0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)",work collection class use video playback record one main class act like public interface methods like play stop pause record etc workhorse class video decode video encode learn existence nest clas...
2,write web service .net app ready consume see numerous examples homegrown code consume service oppose use auto generate methods visual studio create add web reference advantage,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"(1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)",write web service .net app ready consume see numerous examples homegrown code consume service oppose use auto generate methods visual studio create add web reference advantage
3,would like version property application incremented build sure enable functionality visual studio 20052008 try specify assemblyversion 10 get exactly want also use settings file earlier attempt as...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,"(0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)",would like version property application incremented build sure enable functionality visual studio 20052008 try specify assemblyversion 10 get exactly want also use settings file earlier attempt as...
4,simplest way connect query database set record c#,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,"(0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)",simplest way connect query database set record c#


In [16]:
df_merged_new = df_merged[['text','labels']].copy()
df_merged_new.reset_index(drop=True, inplace=True)

In [17]:
df_merged_new.head()

,text,labels
0,little game write c use database back-end trade card game want implement function card script mean essentially interface icard card class implement public class card056 icard contain function call...,"(1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)"
1,work collection class use video playback record one main class act like public interface methods like play stop pause record etc workhorse class video decode video encode learn existence nest clas...,"(0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)"
2,write web service .net app ready consume see numerous examples homegrown code consume service oppose use auto generate methods visual studio create add web reference advantage,"(1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)"
3,would like version property application incremented build sure enable functionality visual studio 20052008 try specify assemblyversion 10 get exactly want also use settings file earlier attempt as...,"(0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)"
4,simplest way connect query database set record c#,"(0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)"


In [18]:
from sklearn.model_selection import train_test_split

# Split the data into train and test sets with a 60:20:20 ratio
df_train, df_val_test = train_test_split(df_merged_new, test_size=0.4, random_state=SEED)
df_val, df_test = train_test_split(df_val_test, test_size=0.5, random_state=SEED)

print(f"df_merged_new shape: {df_merged_new.shape}")
print(f"df_train shape: {df_train.shape}")
print(f"df_val shape: {df_val.shape}")
print(f"df_test shape: {df_test.shape}")


df_merged_new shape: (44501, 2)
df_train shape: (26700, 2)
df_val shape: (8900, 2)
df_test shape: (8901, 2)


In [19]:
with open('/content/drive/MyDrive/thesis/feature_eng_dataframe.pkl', 'wb') as f:
    pickle.dump((df_train,df_val,df_test,tag_cols), f)